## TODO 
### 1) сделать предикты на тестовом датасете и скинуть Саше
### 2) аугментировать точки
### 3) не использовать координаты x,y,z

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
import numpy as np
from sklearn.metrics import recall_score, precision_score, confusion_matrix, accuracy_score
from catboost import CatBoostClassifier
import torch
import torch.nn as nn
import torch.utils.data as data_utils
import torch.nn.functional as F
from sklearn import metrics
import sys
import numpy as np

%matplotlib inline

In [2]:
# df_train = pd.read_csv("train_dataset_train.csv")
df_train = pd.read_csv("file_3/train_dataset_train.csv")
df_test = pd.read_csv("test_dataset_test.csv")

## Рассмотрим датасет по ближе

In [3]:
print(df_train.shape)
print(df_test.shape)

(4222094, 37)
(1809469, 5)


In [4]:
df_train.head(5)

,id,Easting,Northing,Height,Reflectance,Class,feature_1,feature_2,feature_3,feature_4,...,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31
0,2321251,431696.5375,6.032319e+06,69.2226,-11.14,0,-3.3678,0.0151,1281,857,...,0.0011,0.8059,0.5911,0.0339,-0.5919,0.8058,0.0207,-0.0151,-0.0367,0.9992
1,3515173,431710.3835,6.032291e+06,68.9711,-15.16,3,-3.6193,0.0441,180,27,...,0.0276,0.3588,-0.8831,0.3024,-0.8216,-0.1450,0.5514,-0.4431,-0.4463,-0.7775
2,2320295,431696.8099,6.032322e+06,69.2453,-13.59,0,-3.3451,0.0203,1666,1031,...,0.0013,-0.9150,0.4035,0.0027,0.4031,0.9145,-0.0334,-0.0159,-0.0294,-0.9994
3,2454459,431680.4542,6.032343e+06,69.1892,-11.21,0,-3.4012,0.0150,1575,1036,...,0.0002,-0.9998,0.0152,-0.0120,0.0150,0.9997,0.0214,0.0123,0.0212,-0.9997
4,4608150,431720.0914,6.032288e+06,67.3252,-9.10,0,-5.2652,0.0079,495,485,...,0.0001,0.7761,-0.6301,0.0261,-0.6301,-0.7765,-0.0104,0.0268,-0.0084,-0.9996


In [5]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4222094 entries, 0 to 4222093
Data columns (total 37 columns):
 #   Column        Dtype  
---  ------        -----  
 0   id            int64  
 1   Easting       float64
 2   Northing      float64
 3   Height        float64
 4   Reflectance   float64
 5   Class         int64  
 6    feature_1    float64
 7    feature_2    float64
 8    feature_3    int64  
 9    feature_4    int64  
 10   feature_5    float64
 11   feature_6    float64
 12   feature_7    float64
 13   feature_8    float64
 14   feature_9    float64
 15   feature_10   float64
 16   feature_11   float64
 17   feature_12   float64
 18   feature_13   float64
 19   feature_14   float64
 20   feature_15   float64
 21   feature_16   float64
 22   feature_17   float64
 23   feature_18   float64
 24   feature_19   float64
 25   feature_20   float64
 26   feature_21   float64
 27   feature_22   float64
 28   feature_23   float64
 29   feature_24   float64
 30   feature_25   floa

Пострим на распределение данных

In [6]:
# sns.countplot(x = "Class" , data  = df_train).set_title('Распределение класса точки')

In [7]:
# sns.set(rc={'figure.figsize':(18,10)})
# sns.stripplot(data = df_train, x= "Class", y = "Reflectance").set_title('Зависимость класса от параметра отражения');

In [8]:
# plt.rcParams['figure.figsize']=(15,15)

# g = sns.heatmap(df_train.corr(), square = True, annot=True)

## Выделим выборки

In [9]:
df_train = df_train.dropna()

In [10]:
X = df_train.drop(["Class", "id"], axis = 1)
y = df_train[["Class"]]

In [11]:
y = pd.get_dummies(y["Class"])

In [12]:
n_classes = 6

In [13]:
X.shape

(4222094, 35)

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

## Создание модели

In [15]:
train_target = torch.tensor(y_train.values.astype(np.float32))
train = torch.tensor(X_train.values.astype(np.float32)) 
train_tensor = data_utils.TensorDataset(train, train_target) 
train_loader = data_utils.DataLoader(dataset = train_tensor, batch_size = 128, shuffle = True)

test_target = torch.tensor(y_test.values.astype(np.float32))
test = torch.tensor(X_test.values.astype(np.float32)) 
test_tensor = data_utils.TensorDataset(test, test_target) 
test_loader = data_utils.DataLoader(dataset = test_tensor, batch_size = 128, shuffle = False)

In [16]:
print(next(iter(train_loader))[0].shape, next(iter(train_loader))[1].shape)

torch.Size([128, 35]) torch.Size([128, 6])


In [19]:
def get_predictions(loader, model, device):
    model.eval()
    saved_preds = []
    true_labels = []

    with torch.no_grad():
        for x,y in loader:
            x = x.to(device)
            y = y.to(device)
#             print(f"y: {y}")
#             scores = torch.argmax(model(x), dim=1).unsqueeze(1)
            scores = model(x)
#             print(f"scores: {scores, scores.shape}")
            saved_preds += scores.tolist()
            true_labels += y.tolist()
#             print(f"true_labels: {true_labels}\nsaved_preds: {saved_preds}")
#             sys.exit(1)

    model.train()
    return saved_preds, true_labels

class NN(nn.Module):
    def __init__(self, input_size, hidden_dim, num_classes=6):
        super(NN, self).__init__()
        self.bn = nn.BatchNorm1d(input_size)
        self.fc1 = nn.Linear(input_size, hidden_dim//2)
        self.fc2 = nn.Linear(hidden_dim//2, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, num_classes)


    def forward(self, x):
        N = x.shape[0]
        x = self.bn(x)
        x = F.elu(self.fc1(x))
        x = F.elu(self.fc2(x))
        return F.softmax(self.fc3(x))
    
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
model = NN(input_size=35, hidden_dim=100).to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=2e-3, weight_decay=1e-4)
loss_fn = nn.CrossEntropyLoss()

for epoch in range(20):
    probabilities, true = get_predictions(test_loader, model, device=DEVICE)
    print(f"VALIDATION ROC: {metrics.roc_auc_score(true, probabilities)}")
    print("Recall score",recall_score(np.argmax(true, axis=1), np.argmax(probabilities, axis=1), average='micro', zero_division=True))

    for batch_idx, (data, targets) in enumerate(train_loader):
        data = data.to(DEVICE)
        targets = torch.tensor(targets).to(DEVICE)
                
        # forward
        scores = model(data)
#         print(f"scores: {scores}\n targets: {targets}")
        loss = loss_fn(scores, targets)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

/home/anvar/anaconda3/envs/ds/lib/python3.7/site-packages/ipykernel_launcher.py:36: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


VALIDATION ROC: 0.5
Recall score 0.013295921694513548


/home/anvar/anaconda3/envs/ds/lib/python3.7/site-packages/ipykernel_launcher.py:50: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/home/anvar/anaconda3/envs/ds/lib/python3.7/site-packages/ipykernel_launcher.py:36: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


VALIDATION ROC: 0.9491100010003621
Recall score 0.9752642644373373


/home/anvar/anaconda3/envs/ds/lib/python3.7/site-packages/ipykernel_launcher.py:50: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/home/anvar/anaconda3/envs/ds/lib/python3.7/site-packages/ipykernel_launcher.py:36: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


VALIDATION ROC: 0.9464581846762137
Recall score 0.9765100909579679


/home/anvar/anaconda3/envs/ds/lib/python3.7/site-packages/ipykernel_launcher.py:50: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/home/anvar/anaconda3/envs/ds/lib/python3.7/site-packages/ipykernel_launcher.py:36: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


KeyboardInterrupt: 

In [ ]:
true_labels = np.argmax(true, axis=1)

In [ ]:
predicted_labels = np.argmax(probabilities, axis=1)

In [ ]:
print(f"confusion_matrix: {confusion_matrix(true_labels, predicted_labels)}")
print(f"recall_score: {recall_score(true_labels, predicted_labels, average='micro', zero_division=True)}")
print(f"precision_score: {precision_score(true_labels, predicted_labels, average='macro', zero_division=True)}")
print(f"accuracy_score: {accuracy_score(true_labels, predicted_labels)}")